# 02 - Portfolios

In this notebook you can find simple examples that showcase how to programatically create portfolios.

In [2]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [11]:
import time
import logging

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [4]:
srv = Server()
srv.start()

# you can optionally silence the logging output uncommenting the following
# logging.getLogger('ibkr-algotrading').setLevel(logging.WARN)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

First, it is a good idea to know which is our balance,

In [5]:
srv.balance

(1000000.0, 'USD')

> NOTE: with a simulated IBKR Paper Account you start with 1'000'000 USD, which you can use to test out trading.

## Create Portfolio

we can now request some stocks and create a portfolio with them,

In [6]:
stocks = srv[['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA'], '1y', '1d']

portfolio = Portfolio(wealth=100, stocks=stocks)

[2021-08-28 16:41:30,711] I T140420823979584 server.py:298: Started StockCache thread for bar "1d". 
		Next data update will be in  33509.29s.


The `wealth` argument is in the same currency as our account's balance.

We can now check how much fraction of stock is allocated for each of the stocks.

Of course, right now we haven't bought any stock so it's all zero,

In [7]:
portfolio.alloc

array([0., 0., 0., 0., 0.])

Similarly, our distribution of wealth accross the stocks looks like an all-zero array, and
this is the only time it will be like so before we perform our first order.

In any other case the "weights" will sum up to 1.

In [8]:
portfolio.weights

array([0., 0., 0., 0., 0.])

we finally add the portfolio to the server so it knows about its existance,

In [9]:
srv.add_portfolio(portfolio)

## Place an order on our portfolio

To prevent accidentally losing all of our money, we will trade under a simulated context in which orders are only previewed but we still get whole information on what would happen,

In [21]:
weights = np.ones(5) / 5.0

with srv.simulated():
    portfolio.order(weights)

[2021-08-28 17:32:13,927] W T140421889238848 server.py:524: ENTERING SIMULATED CONTEXT. ALL FOLLOWING TRANSACTIONS WILL BE SIMULATED.
[2021-08-28 17:32:14,538] W T140420832372288 instruments.py:132: Provided quantity for stock "Stock("SPY", conid=756733)" is too precise. The following amount will be lost: 2.2768245622195593e-18
[2021-08-28 17:32:14,538] W T140420815586880 instruments.py:132: Provided quantity for stock "Stock("XOM", conid=13977)" is too precise. The following amount will be lost: 2.031523820694714e-17
[2021-08-28 17:32:14,540] W T140420583511616 instruments.py:132: Provided quantity for stock "Stock("MSFT", conid=272093)" is too precise. The following amount will be lost: 2.8053731213062427e-18
[2021-08-28 17:32:14,540] W T140420600297024 instruments.py:132: Provided quantity for stock "Stock("AAPL", conid=265598)" is too precise. The following amount will be lost: 1.2170169386149787e-17
[2021-08-28 17:32:14,543] W T140420591904320 instruments.py:132: Provided quantity

ValueError: total != commission + stock. total: 20.0 | commission: 0.2 | stock: 19.81